# WoW Classic Druid Feral Tank Attack Speed Analysis by Zidnae

*Last updated 7/25/2020*
## A Quantitative Look at the Effect of Attack Speed Buffs on Feral Tanking in WoW Classic

The base druid attack speed in bear form is 2.5. Throughout the game there are a number of items and buffs that increase attack speed. We can calculate the cumulative effect of these attack speed buffs using the following [formula](https://classic-wow.fandom.com/wiki/Attack_speed)

```
Attack_speed = "current attack speed" / (("Percent increase or decrease" / 100) + 1 )
```

Available items/effects that increase attack speed are:

| Item   |      Attack Speed Increase      |  Type |
|----------|-------------|--------|
| [Iron Counterweight](https://classic.wowhead.com/item=6043/iron-counterweight) |  3% | weapon enchant |
| [Libram of Rapidity](https://classic.wowhead.com/quest=7483/libram-of-rapidity) |    1% | head/leg enchant   |  
| [Manual Crowd Pummeler](https://classic.wowhead.com/item=9449/manual-crowd-pummeler) | 50%| on-use weapon effect |
| [Kiss of the Spider](https://classic.wowhead.com/item=22954/kiss-of-the-spider) | 20% |on-use trinket |
| [Warchief's Blessing](https://classic.wowhead.com/spell=16609/warchiefs-blessing) | 15%| world buff |
| [Enchant Gloves Minor Haste](https://classic.wowhead.com/spell=13948/enchant-gloves-minor-haste) | 1% | glove enchant |
| [Gnomish Battle Chicken](https://classic.wowhead.com/spell=12906/gnomish-battle-chicken) | 5% | "Battle Squak" buff occasionally given by engineering pet to party |

There's also the JuJu Flurry buff but it's not included here since the effect was [bugged in Vanilla](https://eu.forums.blizzard.com/en/wow/t/juju-flurry-buff-%E2%80%93-not-a-bug/117855) and the bug was replicated in Classic. Also the Battle Chicken buff will be omitted from the analysis below since it's difficult to consistently get the effect to proc. 


Common combinations of the above attack speed items yield attack speeds (according to [WowWiki](https://classic-wow.fandom.com/wiki/Attack_speed) attack speed values get rounded to 4 significant digits)

In [1]:
ic = 1.03
libram = 1.01
mcp = 1.50
kots = 1.20
wcb = 1.15
ecmh = 1.01

attackSpeedCombinations = {
    "Base Attack Speed" : 2.5,
    "Librams + Glove Enchant" : round(2.5 / (libram * libram *ecmh),4),
    "Iron Counterweight + Librams + Glove Enchant" : round(2.5 / (ic * libram * libram *ecmh),4),
    "Iron Counterweight + Librams + Glove Enchant + Warchief's Blessing" : round(2.5 / (ic * libram * libram *ecmh * wcb),4),
    "Manual Crowd Pummeler" : round(2.5 / (mcp),4),
    "Manual Crowd Pummeler + Librams + Glove Enchant" : round(2.5 / (mcp * libram * libram *ecmh),4),
    "Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant" : round(2.5 / (mcp * ic * libram * libram *ecmh),4),
    "Manual Crowd Pummeler + Librams + Glove Enchant + Warchief's Blessing" : round(2.5 / (mcp * libram * libram *ecmh *wcb),4),
    "Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant + Warchief's Blessing" : round(2.5 / (mcp * ic * libram * libram *ecmh *wcb),4),
    "Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant + Kiss of the Spider" : round(2.5 / (mcp * ic * libram * libram *ecmh *kots),4),
    "Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant + Kiss of the Spider + Warchief's Blessing" : round(2.5 / (mcp * ic * libram * libram *ecmh *kots * wcb),4)
}
justEnchants ={
    "Base Attack Speed" : attackSpeedCombinations["Base Attack Speed"],
    "Librams + Glove Enchant" : attackSpeedCombinations["Librams + Glove Enchant"],
}
icAndEnchants ={
    "Base Attack Speed" : attackSpeedCombinations["Base Attack Speed"],
    "Iron Counterweight + Librams + Glove Enchant"  : attackSpeedCombinations["Iron Counterweight + Librams + Glove Enchant"],
}
allianceOnlyAttackSpeedCombinations = {k:v for k,v in attackSpeedCombinations.items() if "Warchief's Blessing" not in k}
attackSpeedCombinationsNoKiss = {k:v for k,v in attackSpeedCombinations.items() if "Kiss" not in k}
allianceOnlyAttackSpeedCombinationsNoKiss = {k:v for k,v in allianceOnlyAttackSpeedCombinations.items() if "Kiss" not in k}
table = "| Items   |      Attack Speed |\n|:---------|------------|\n"
for name, value in attackSpeedCombinations.items():
    table += f"| {name} | {value} |\n"

from IPython.display import display, Markdown, Latex
display(Markdown(table))

| Items   |      Attack Speed |
|:---------|------------|
| Base Attack Speed | 2.5 |
| Librams + Glove Enchant | 2.4265 |
| Iron Counterweight + Librams + Glove Enchant | 2.3558 |
| Iron Counterweight + Librams + Glove Enchant + Warchief's Blessing | 2.0485 |
| Manual Crowd Pummeler | 1.6667 |
| Manual Crowd Pummeler + Librams + Glove Enchant | 1.6177 |
| Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant | 1.5705 |
| Manual Crowd Pummeler + Librams + Glove Enchant + Warchief's Blessing | 1.4067 |
| Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant + Warchief's Blessing | 1.3657 |
| Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant + Kiss of the Spider | 1.3088 |
| Manual Crowd Pummeler + Iron Counterweight + Librams + Glove Enchant + Kiss of the Spider + Warchief's Blessing | 1.1381 |


# Spell Batching 

An important aspect to consider when analyzing attack speed in classic is spell batching. Quoting directly from https://github.com/magey/classic-warrior/wiki/Spell-batching#auto-attacks

> ## Official statements from Blizzard
> Source: https://web.archive.org/web/20140619092137/us.battle.net/wow/en/forum/topic/13087818929?page=6#114
> > **Any action that one unit takes on another different unit used to be processed in batches every 400ms**. Some very attentive people may have noticed that healing yourself would give you the health instantly (minus client/server latency), whereas healing another unit would incur a delay of between 0ms and 400ms (again, on top of client/server latency). Same with damaging, applying auras, interrupting, knocking back, etc.
>
> Source: https://eu.forums.blizzard.com/en/wow/t/spell-batching-in-classic/39542
>
> > For WoW Classic, we’re moving spell casts to a low-priority loop that will cause them to be processed at the frequency that best fits how the game actually played in version 1.12. Two mages will be able to Polymorph each other somewhat reliably, resulting in two sheep 
nervously pacing around at range. Two warriors will be able to Charge one another, and the end result will be both warriors standing stunned in each other’s original location.
>
> ## Observations from the beta
> The key takeaway from Blizzard's statements regarding spell batching is "**Any action that one unit takes on another different unit used to be processed in batches every 400ms**". What does this mean in practice? first of all it means that any action that you take on yourself (healing yourself, buffing yourself, etc.) is instant; only actions taken on other units, be it players or mobs, are batched.
>
> It's important to note that only the application of damage or healing and buffs/debuffs resulting from an attack, cast or use of an ability are delayed until the next batch processing window; the calculations which determine if the attack hit or missed and if it triggered any procs happen **immediately** when the cast finishes, and the player can immediately start casting another spell or use another ability before the damage of the previous one has even been applied (you can find an example of this below involving Frostbolt).
>
>
> ### Auto attacks
When you're auto attacking a target the combat log will show you a `SWING_DAMAGE` message if you hit the target or a `SWING_MISSED` message if you missed or were avoided at the interval of your swing speed, i.e. if you're using a 3.6 speed weapon then these messages will show up every 3.6 seconds. But these messages are only one part of the equation; they signal when the attack table roll happens and damage calculations are performed but the actual damage of your auto attack is not applied to the target until the next batch is processed by the server. When this happens, and advanced combat logging is enabled in the client, the combat log will show a `SWING_DAMAGE_LANDED` message signaling the actual application of the damage on the target. Example (edited for brevity):
>
> ```
6/8 11:38:15.866  SWING_DAMAGE (Glance for 125)
6/8 11:38:15.967  SWING_DAMAGE_LANDED (Glance for 125)
6/8 11:38:19.480  SWING_DAMAGE (Glance for 107)
6/8 11:38:19.589  SWING_DAMAGE_LANDED (Glance for 107)
6/8 11:38:23.089  SWING_MISSED (Miss)
6/8 11:38:26.684  SWING_MISSED (Miss)
6/8 11:38:30.290  SWING_DAMAGE (Hit for 134)
6/8 11:38:30.528  SWING_DAMAGE_LANDED (Hit for 134)
6/8 11:38:33.887  SWING_MISSED (Parry)
6/8 11:38:37.409  SWING_DAMAGE (Crit for 258)
6/8 11:38:37.814  SWING_DAMAGE_LANDED (Crit for 258)
```
>
> If you look at the time difference between the different `SWING_DAMAGE` and `SWING_MISSED` messages you will see they align with a 3.6 weapon swing timer.



# Attack Speed Visualizations
Armed with this information we can now visualize the effects of the attack speed buffs on when our auto attacks land.
- It's assumed that you begin attacking precisely after the fisrt spell batch
- Kiss of the Spider is used on cooldown
- Maximum Manual Crowd Pummelers uptime, 3 puemmler charges are expended back to back and afterwards a new Pummeler is equipped and after the 30 second cooldown is activated

### NOTE: Click on the Legend to Enable/Disable lines on the graph


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import WheelZoomTool
import warnings
from bokeh.models.annotations import Title

output_notebook()

SPELL_BATCH_INTERVAL = .400 #secs
THREAT_PER_HIT = 100

def getSpellBatchTimes(encounterTime):
    return np.arange(0, encounterTime, SPELL_BATCH_INTERVAL)

def computeThreat(attackTimes,encounterTime):
    spellBatchTimes = np.arange(0, encounterTime*2, SPELL_BATCH_INTERVAL)
    threat = np.array(range(1,len(attackTimes)+1)) * THREAT_PER_HIT
    threat = []
    threatCeil = 0
    for t in spellBatchTimes:
        if t in attackTimes:
            threatCeil += THREAT_PER_HIT
        threat.append(threatCeil)
    return threat

def computeAttackTimes(combatStartTime, attackSpeed, encounterTime,kissOfTheSpider=False,manualCrowdPummeler=False):
    spellBatchTimes = np.arange(0, encounterTime*2, SPELL_BATCH_INTERVAL)
    attackTimesWithoutSpellBatch = [combatStartTime]
    while max(attackTimesWithoutSpellBatch) < encounterTime:
        nextAttack = attackSpeed
        if kissOfTheSpider and max(attackTimesWithoutSpellBatch) % 120 > 15:
            nextAttack += attackSpeed * .2
        if manualCrowdPummeler  and max(attackTimesWithoutSpellBatch) % 120 > 90:
            nextAttack += attackSpeed *.5
        attackTimesWithoutSpellBatch.append(max(attackTimesWithoutSpellBatch)+nextAttack)
    #attackTimesWithoutSpellBatch = np.arange(combatStartTime, encounterTime+5, attackSpeed)
    spellBatchedAttacks = []
    for attackTime in attackTimesWithoutSpellBatch:
        attackOnBatch = spellBatchTimes[spellBatchTimes >= attackTime][0]
        if attackOnBatch > encounterTime:
                spellBatchedAttacks.append(encounterTime)
                break
        spellBatchedAttacks.append(attackOnBatch)
    spellBatchedAttacks=np.array(spellBatchedAttacks)
    return spellBatchedAttacks

from bokeh.palettes import viridis
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.models.tools import HoverTool

def generateBokehPlot(attackSpeedCombos, encounterTime,combatStartTime=0.00,title=''):
    p = figure(plot_width=1800, plot_height=400)

    colors = viridis(len(attackSpeedCombos))
    if len(attackSpeedCombos) == 2:
        colors = viridis(4)
    for (itemCombination, attackSpeed),color in list(zip(attackSpeedCombos.items(),colors))[::-1]:
        attackTimes = computeAttackTimes(combatStartTime, attackSpeed, encounterTime,kissOfTheSpider='Kiss' in itemCombination,manualCrowdPummeler='Pummeler' in itemCombination)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            source = ColumnDataSource(data=dict(
                time=getSpellBatchTimes(encounterTime)[1:],
                threat=computeThreat(attackTimes,encounterTime)
            ))
        p.step(
                'time','threat',
                source=source,
                color=color, alpha=0.8, legend_label=itemCombination,
            line_width=2
            
        )

    p.legend.location = "top_left"
    p.legend.click_policy="hide"
    t = Title()
    t.text = title
    t.align = 'center'
    t.text_font_size = '20px'
    p.title = t
    p.yaxis.axis_label = 'Threat'
    p.xaxis.axis_label = 'Encounter Time (secs)'
    p.add_layout(p.legend[0], 'right')
    return p

Loading BokehJS ...

In [3]:
#generatePlot(allianceOnlyAttackSpeedCombinations, 15)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
p = generateBokehPlot(allianceOnlyAttackSpeedCombinations, 15,title='15 Second Encounter')
show(p)
p = generateBokehPlot(allianceOnlyAttackSpeedCombinations, 30,title='30 Second Encounter')
show(p)
p = generateBokehPlot(allianceOnlyAttackSpeedCombinations, 60,title='60 Second Encounter')
show(p)
p = generateBokehPlot(allianceOnlyAttackSpeedCombinations, 120,title='120 Second Encounter')
show(p)
p = generateBokehPlot(allianceOnlyAttackSpeedCombinations, 240,title='240 Second Encounter')
show(p)

## With Warchiefs Blessing
The above plots ommited Warchief's Blessing to reduce visual clutter. With Warchief's Blessing the plots look like

### NOTE: Click on the Legend to Enable/Disable lines on the graph

In [4]:
p = generateBokehPlot(attackSpeedCombinations, 15,title='15 Second Encounter')
show(p)
p = generateBokehPlot(attackSpeedCombinations, 30,title='30 Second Encounter')
show(p)
p = generateBokehPlot(attackSpeedCombinations, 60,title='60 Second Encounter')
show(p)
p = generateBokehPlot(attackSpeedCombinations, 120,title='120 Second Encounter')
show(p)
p = generateBokehPlot(attackSpeedCombinations, 240,title='240 Second Encounter')
show(p)

# Differing Combat Start Times
The above plots assumed that you began attacking right after the first spell batch. We can alter the time during the spell batch you get into range and register the first to how it will impact when your attacks will land.

Only the base attack speed and the 3% haste from the librams and glove enchant are plotted to keep the comparion between the different starting times more apparent.
## Starting 100ms into a Spell Batch


In [5]:
p = generateBokehPlot(justEnchants, 15,combatStartTime=.1,title='Attack Times when Starting to Attack 100ms into a Spell Batch')
show(p)
p = generateBokehPlot(justEnchants, 15,combatStartTime=.2,title='Attack Times when Starting to Attack 200ms into a Spell Batch')
show(p)
p = generateBokehPlot(justEnchants, 15,combatStartTime=.3,title='Attack Times when Starting to Attack 300ms into a Spell Batch')
show(p)

The takeaway from these plots is that there isn't some fixed X seconds into a fight when you start to reap the benefits from haste. The benefits of haste are not only a function of the fight duration but also when the spell batches occur during the fight. 

# Another Way to Think About Haste
What I can say is that your base attack speed is 2.5 and with a 6% attack speed buff (the Iron Counterweight and attack speed enchants) your attack speed is 2.3558. Given the spell batch interval in Classic is 400ms we have the following

In [6]:
deltaAttackSpeed = round(2.5 - 2.3558,4)
print(f'Attack Speed Difference:{deltaAttackSpeed}')
numberOfAttacksTillNextBatch = round(0.400 / deltaAttackSpeed,4)
print(f'Number of Attacks Until You Advance a Spell Batch:{numberOfAttacksTillNextBatch}')
print(f'Time elapsed till you advance a spell batch about {numberOfAttacksTillNextBatch* 2.3558} seconds')

Attack Speed Difference:0.1442
Number of Attacks Until You Advance a Spell Batch:2.7739
Time elapsed till you advance a spell batch about 6.534753619999999 seconds


Put simply, with 6% the attack speed enchants about every 3 auto attacks (or 6.5 seconds) your attack comes out one spell batch sooner than a feral druid without the attack speed enchants. And this effect accumluates over time so after 6 auto attacks you attacks are coming out 2 spell batches sooner than another feral druid and after 9 auto attacks 3 spell batches sooner. 

Repeating this analysis for when you are using a Manual Crowd Pummeler yields

In [7]:
deltaAttackSpeed = round(1.6667 - 1.5705,4)
print(f'Attack Speed Difference:{deltaAttackSpeed}')
numberOfAttacksTillNextBatch = round(0.400 / deltaAttackSpeed,4)
print(f'Number of Attacks Until You Advance a Spell Batch:{numberOfAttacksTillNextBatch}')
print(f'Time elapsed till you advance a spell batch about {numberOfAttacksTillNextBatch* 1.5705} seconds')

Attack Speed Difference:0.0962
Number of Attacks Until You Advance a Spell Batch:4.158
Time elapsed till you advance a spell batch about 6.530139000000001 seconds


With a Manual Crowd Pummeler equiped and the attack speed enchants about every 4 autoattacks (or 6.5 seconds) your auto attacks come out one spell batch sooner than a feral druid without the attack speed enchants.

# FAQ

## Should I Use Haste Enchants?
It is widely accpeted in the feral druid theorycrafting community that the haste enchants, comprising of the librams, glove enchant, and iron counterweight, are the best in slot enchants in terms of expected damage output over a fight of any duration and as a result provide the more value than the competeting enchants in Phase 4.

In Phase 5 you may consider using either the 2% threat enchant for more threat per second than the haste enchant or the 15agi glove enchant for less damage but more mitigation making it a more well rounded choice. In particular 2% threat is recommended on the Horde side since threat is tighter without Blessing of Salvation and 15agi is recommended on the Alliance side as it benefits from Blessing of Kings and threat is less tight.

## Doesn't a fight have to last X seconds in order for me to get an "extra attack" to benefit from haste?

Not true. Consider the plot below. As you can see if a spell batches occurs at 2.4 and 2.8 seconds with the 6% haste enchant your 2.358 attack speed attack gets batched to land at 2.4s while a 2.5s attack speed attack would get batched to land at 2.8s. So you can reap the benefits from haste almost immeadiately after a fight starts. Were to fight end at any of the times where the light purple line is above the dark purple line you would have landed one additional attack than you otherwise would have. And as you can see the portions of time where you would have landed an additional attack grows as the fight duration increases.


In [8]:
p = generateBokehPlot(icAndEnchants, 30,title='30 Second Encounter')
show(p)


## Well what if a fight ends at precisely Y seconds at a time when you don't benefit from haste? Wouldn't that make the other enchants better in this instance?
It's almost impossible to determine when exactly a fight will end. As a result the feral druid theorycrafting community values haste as it's expected damage output over a fight of any duration. 

Refer to the [following proof](https://discord.com/channels/253205420225724416/253206574787461120/726555105532575774) for a more formal look at how to value haste over a fight of any duration.

## Sure, but let's just say for arguments sake a short fight ends at precisely Y seconds at a time when you don't benefit from haste? Would the other enchants be better?
Not necesarily. You don't need have an additional attack to land in order to reap benefits from haste as a tank. Say for example looking at the plot above the fight ends at 20sec, a time when you would not have landed an additional attack due to haste. Even though you didn't land an additional attack and do more damage than you otherwise would have your attacks were still coming out faster than they otherwise would have. That means as a tank you had a higher threat ceiling for portions of the fight than you otherwise would have which is value added as a tank. Every instance when the light purple line is above the dark purple line are times when you had a higher threat ceiling than you otherwise would have had. In this sense even if you don't sneak in an additional attack from haste over the duration of a short fight it's still providing value by giving more room to the DPS to go ham.

### Click Here to Show Code

In [9]:
%%html
<style id=hide>div.input{display:none;}</style>
<button type="button" 
onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">
Show code</button>